# Processing Files using Iteration, Lists, Arrays, and Pandas DataFrames

Let's say we need to load a file of data about rainfall from a CSV file and calculate the mean, min, max, and median rainfall from it.  What are the options we could use?

## Iterating Over Rows to Analyze Data

First let's use the methods we learned to iterate through the rows of a file in order to process each row, one at a time.  We will load it via the csv library and open the file and iterate through the rows to compute the mean, max, and min values of rainfall.

In [ ]:
import csv
with open('data/rain.csv', 'r') as csvfile:
    
    # initialize a counter and variables to contain our descriptive stats
    count = 0 #at the end, divide cumulative_sum by this to get the mean
    cumulative_sum = 0 #our rolling sum
    max_value = -1 #pick a really small number that's guaranteed to be less than the max
    min_value = 1000
    
    # open the file and skip the header row
    my_csv = csv.reader(csvfile)
    next(my_csv)
    
    # loop through each data row
    for row in my_csv:
        
        # rainfall amount is in column 1, only process this row's value if not an empty string
        if not row[1] == '':
            
            # increment the counter and extract this row's rainfall as a float
            count = count + 1
            rainfall = float(row[1])
            
            # add this row's rainfall to the cumulative sum
            cumulative_sum = cumulative_sum + rainfall
            
            # if this row's rainfall is greater than the current max value, update with the new max
            if rainfall > max_value:
                max_value = rainfall

            # if this row's rainfall is less than the current min value, update with the new min    
            if rainfall < min_value:
                min_value = rainfall

    # after looping through all the rows, divide the cumulative sum by the count and round to get the mean
    mean_value = round(cumulative_sum / count, 1)
    
    # print out the mean and max values
    print('mean:', mean_value, 'inches')
    print('max:', max_value, 'inches')
    print('min:', min_value, 'inches')

#### Thought experiment: 
Can you think of a straightforward way to get the median value of rainfall?

## Using Lists to Analyze Data

Some things that are hard in the row iteration approach above, become easier if we can keep all of the values in one object, like a list.

Let's revisit the problems above, by assembling the values from each row into a single list.

In [ ]:
with open('data/rain.csv', 'r') as csvfile:
    x = []
    itemreader = csv.reader(csvfile)
    # Skip the header row
    next(itemreader)
    for row in itemreader:
        # keep only non-missing values
        if row[1] != '':
            x.append(float(row[1]))
x

X is now a single list object with all the values in the file, whereas initially we just had each row producing one list with one element in it (from one row), and then printing that, before recreating it with the value from the next row.  The iteration approach kept only one row at a time, and we could not easily do calculations like a median.

Using the list (x), we should now have an easier time with mean and median calculations, using list methods like sum and len, and sort:

In [ ]:
mean = round(sum(x)/len(x),1)
print(mean)

We can solve the median problem (approximately) by sorting the list, and getting the value that is halfway through the list:

In [ ]:
x.sort()
median = x[int(len(x)/2)]
print(median)

It is also simple to get the min and max, using built-in list indexes to get the first and last element from the sorted list:

In [ ]:
min = x[0]
min

In [ ]:
max = x[-1]
max

OK, this is progress.  Can we now do other math on the data, like multiply each value by 5?

In [ ]:
y = x*5
print(y)

Not what we wanted. That just concatenated 5 copies of the list together!

Let's take a different approach.

In [ ]:
y = []
for item in range(len(x)):
    y.append(x[item]*5)
y

Alright, this is a big improvement over the iteration over rows approach, but it is still a bit tedious.

## Using Numpy to Analyze Data

Now let's review the use of arrays, the basic data structure added by Numpy.

We can convert our list (once we have iterated through the file to create it) to a Numpy array:

In [ ]:
import numpy as np

rain = np.array(x)
rain

In [ ]:
type(rain)

In [ ]:
rain.dtype

A lot of mathematical methods are readily built in to Numpy.  Here are a few for mean, standard deviation, variance, median, min, max, sum.

In [ ]:
round(np.mean(rain),1)

In [ ]:
np.std(rain)

In [ ]:
np.var(rain)

In [ ]:
np.median(rain)

In [ ]:
np.min(rain)

In [ ]:
np.max(rain)

In [ ]:
np.sum(rain)

Much easier than coding for loops and counters, don't you think?  And a lot faster on large datasets, too.

Numpy also integrates very well with Matplotlib to generate charts.  Let's look at a sorted list of the elements in the array.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(np.sort(rain))

# Introducing Pandas

We have moved from processing file with iterators over rows, to lists with iterators, to using Numpy arrays to do vectorized operations that are much faster and are also less complex to read, understand and to code. We moved from operating on one item in a list at a time, to operating on a whole array at one time. Now we want to be able to move to handling the whole table of data at once.

One of the problems we did not attempt to deal with using arrays, or lists, was how to keep rows of data together so that if we skip a missing value in one variable like price, it does not cause the other variables to be mis-aligned due to changes in the length of the array or list for one entry. This is one of many things that the pandas library does for us.
We skip forward a bit to Chapter 6 in Python for Data Analysis in order to learn how to load our data using pandas. Here is how:

In [ ]:
import pandas as pd
df = pd.read_csv('data/rain.csv')
df

Notice a few things that have happened here.

1. The pd.read_csv has enough built-in smarts to read the first row of the file, get the variable names from it.
1. It then read all rows in the file, and used them to create a pandas DataFrame, which is like a set of Numpy arrays we can treat as a table.
1. It created an automatic unique index, beginning with zero.
1. It inferred the type of each variable from the data. 

Let's explore this pandas DataFrame to learn some of its features.  Note that a pandas Series is like one column of this DataFrame, coupled with its own index column.  So the main difference between a Series and a DataFrame is that the latter has multiple columns. Columns can be of different data types, but within a column, must be consistent.

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

We can select subsets of the rows by indexing, and select specific columns by their name:

In [ ]:
df['rainfall_inches'][:6]

We can get all of our statistics on the rainfall_inches column in one short command:

In [ ]:
df['rainfall_inches'].describe()

Notice how it silently handled the missing value for September and gave the correct statistical results?

You can also get these values 'a la carte'.  You might recognize that these are essentially Numpy functions, but that in pandas we can now deal with multiple data types.  Notice that there is some flexibility in syntax (multiple ways go ge some things done).

In [ ]:
df['rainfall_inches'].min()

In [ ]:
df.rainfall_inches.min()

In [ ]:
min(df.rainfall_inches)

In [ ]:
df['rainfall_inches'].max()

In [ ]:
df['rainfall_inches'].median()

We can also do some plotting of the data without much effort:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df.plot(y = 'rainfall_inches', x = 'month_2014')

Check-in time.  How would you compute the sum of rainfall over the year? http://bitly.com/cp255

### Slicing and indexing using loc and iloc

There are two indexing methods you can use to get subsets of rows and columns in a dataframe.  **loc** uses the values of the indices and includes the last value, whereas **iloc** uses the index positions and like usual Python indexing, does not include the last value. 

In [ ]:
df.loc[0:6,:'rainfall_inches']

In [ ]:
df.iloc[0:6,0:2]

Note also that the 0th index position for columns is not the index column, but the first column of data in the dataframe, and the same applies for rows.

In [ ]:
df.iloc[0:6,1:2]

### Filtering on values

You can easily filter a dataframe for one or more conditions based on the values in a column. Below we filter df to select only months with less than 4 inches of rainfall.  

In [ ]:
df[df['rainfall_inches'] < 4]

Notice the nested use of df.  What happens if you don't do that?

In [ ]:
df['rainfall_inches'] < 4

You can also select rows based on the values of more than one column.  Just remember to nest the individual conditions within parentheses:

In [ ]:
df[(df['month_2014']=='jan') & (df['rainfall_inches'] > 5)]

### Using string functions to filter a dataframe

We will get more into string functions in the next session, based on content in Chapter 7 of Python for Data Analysis.  But here is a preview of how you can use a string function applied to a column of data to filter the data.  Notice the **str** component of the syntax.

In [ ]:
df[df['month_2014'].str.contains('j')]

You can even do statistics on such a filtered set:

In [ ]:
df[df['month_2014'].str.contains('j')]['rainfall_inches'].mean()

### Missing data

This filtering approach is handy when you want to eliminate missing data also.

In [ ]:
df[df['rainfall_inches'].notnull()]

Let's say we find the value for sep should be 2.5. How could we set that value using the loc indexing?

In [ ]:
df.loc[8,'rainfall_inches'] = 2.5
df

### Sorting

In [ ]:
df.sort_values(['rainfall_inches'], ascending = False)

### Getting unique value lists and counts

In [ ]:
df['month_2014'].unique()

In [ ]:
df['month_2014'].count()

## Working with Pandas -- an in-class working session

OK, let's create a DataFrame from a dictionary, following the example on pg 129 of Python for Data Analysis (PDA).

In [ ]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 'year': [2000, 2001, 2002, 2001, 2002],
'pop': [1.5, 1.7, 3.6, 2.4, 2.9]}
sp = pd.DataFrame(data)

Explain the contents and structure of 'data'

What does 'pd.DataFrame(data)' do? What if we did not begin that line with 'sp ='?

Look at the contents of sp, using just sp by itself, and 'print sp'.  

In [ ]:
sp

In [ ]:
print(sp)

We can refer to a column in two ways:

In [ ]:
sp['state']

In [ ]:
sp.state

Generally it is preferable to be explicit and use the df['column'] approach.

We can step through the rows in a dataframe

In [ ]:
for label in sp.state.index:
    print(sp.state[label])

And find the index value within each entry of a specific substring

In [ ]:
for label in sp.state.index:
    print(sp.state[label].find('io'))

In [ ]:
for label in sp.state.index:
    if sp.state[label]=='Ohio':
        print(sp.state[label])
    else:
        print('Not Ohio')

## Now Your Turn

Below are a series of questions, with the answers remaining for you to fill in by using pandas expressions that draw on the methods in Chapter 5.  You should not need to use anything more than the content of this chapter -- a subset of the methods summarized above, to do this exercise.  Hopefully you can complete it within class if you've been keeping up with the reading.

How can we get a quick statistical profile of all the numeric columns?

Can you get a profile of a column that is not numeric, like state? Try it.

How can we print the data types of each column?

How can we print just the column containing state names?

How can we get a list of the states in the DataFrame, without duplicates?

How can we compute the mean of population across all the rows?

How can we compute the maximum population across all the rows?

How can we compute the 20th percentile value of population? 

How can we compute a Boolean array indicating whether the state is 'Ohio'?

How can we select and print just the rows for Ohio?

How can we create a new DataFrame containing only the Ohio records?

How can we select and print just the rows in which population is more than 2?

How could we compute the mean of population that is in Ohio, averaging across years?

How can we print the DataFrame, sorted by State and within State, by Population?

How can we print the row for Ohio, 2002, selecting on its values (not on row and column indexes)?

How can we use row and column indexing to set the population of Ohio in 2002 to 3.4?